In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from ephysvibe.trials.spikes import firing_rate,sp_constants
# from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.task import def_task,task_constants
from collections import defaultdict
from typing import Dict
from ephysvibe.structures.spike_data import SpikeData
from ephysvibe.structures.bhv_data import BhvData
from ephysvibe.analysis import circular_stats
import os 
seed = 2023

### Def functions

In [2]:
def moving_average(data:np.ndarray,win:int,step:int=1)-> np.ndarray:
    d_shape=data.shape
    count = 0
    if len(d_shape) == 3:
        d_avg = np.zeros((d_shape[0],d_shape[1],int(np.floor(d_shape[2]/step))))
        for i_step in np.arange(0,d_shape[2]-win,step):
            d_avg[:,:,count] = np.mean(data[:,:,i_step:i_step+win],axis=2)
            count +=1
    if len(d_shape) == 2:
        d_avg = np.zeros((d_shape[0],int(np.floor(d_shape[1]/step))))
        for i_step in np.arange(0,d_shape[1]-win,step):
            d_avg[:,count] = np.mean(data[:,i_step:i_step+win],axis=1)
            count +=1
    if len(d_shape) == 1:
        d_avg = np.zeros((int(np.floor(d_shape[0]/step))))
        for i_step in np.arange(0,d_shape[0]-win,step):
            d_avg[count] = np.mean(data[i_step:i_step+win],axis=0)
            count +=1
    return d_avg

In [3]:
def get_task_variables(data:SpikeData,bhv:BhvData,in_out:str='in'):
    # Select trials (correct and DMTS task) and create task frame
    trial_idx = np.where(np.logical_and(data.trial_error == 0, data.block == 1))[0]

    if np.any(np.isnan(data.neuron_cond)):
        neuron_cond = np.ones(len(data.clustersgroup))
    else:
        neuron_cond=data.neuron_cond
    task = def_task.create_task_frame(
        condition=bhv.condition[trial_idx],
        test_stimuli=bhv.test_stimuli[trial_idx],
        samples_cond=task_constants.SAMPLES_COND,
        neuron_cond = neuron_cond,
    )
    task = task[task['in_out']==in_out]
    return task, trial_idx

In [4]:
def delete_match(test_stimuli,code):
    code = (code-1).astype(int)
    tests_shape=test_stimuli.shape
    test_stimuli = np.concatenate([test_stimuli,np.ones((tests_shape[0],1))],axis=1)
    test_stimuli[np.arange(tests_shape[0]),code] =  np.nan
    test_stimuli = test_stimuli[:,:tests_shape[1]]
    return test_stimuli.astype(str)

In [5]:
def get_sp_feature(sp_samples_samp,test_stimuli_samp,code_samples_samp,color_orient,stim_num):
    all_sp_tests=[]
    st = 0
    end =1
    if color_orient == 1:
        st = 1
        end =2
    for i_num,i_stim in enumerate(stim_num):
        i_trial, i_test = np.where(np.char.find(test_stimuli_samp,i_stim,start=st, end=end)==color_orient)
        tests_on = code_samples_samp[i_trial,2*i_test+6].astype(int)
        sp_stim = sp_samples_samp[i_trial]
        sp_tests = SpikeData.indep_roll(sp_stim, -tests_on, axis=1)[:, 0:550]
        if np.isnan(np.sum(sp_tests)):
            raise ValueError('nan values')
        all_sp_tests.append(sp_tests)
    return all_sp_tests

In [6]:
## Permutation test
def permutation_test(mean_select,mean_null):
    radius = mean_select[:,0]
    angle =  mean_select[:,1]
    X = (np.array(radius) * np.cos(angle))
    Y = (np.array(radius) * np.sin(angle))
    ampl_dir_vector = np.sqrt(X**2+Y**2)
    radius = mean_null[:,0]
    angle =  mean_null[:,1]
    X = (np.array(radius) * np.cos(angle))
    Y = (np.array(radius) * np.sin(angle))
    ampl_null_vector = np.sqrt(X**2+Y**2)
    diff=[]
    for i in range(len(ampl_dir_vector)):
        # rotate vecto to compare all values with the null vector
        shift = np.concatenate([ampl_dir_vector[i:],ampl_dir_vector[:i]])
        diff.append(shift-ampl_null_vector)
    diff=np.concatenate(diff)
    p_value = np.sum(diff<0)/len(diff)
    return p_value

In [7]:
def get_null_vector_from_samples(trials_fr,min_n_trials,seed:int=1,n_iterations:int=1000):
    rng = np.random.default_rng(seed=seed)
    it_seed = rng.integers(low=1, high=10000, size=n_iterations, dtype=int)
    stim_num=['1','2','3','4','5','6','7','8']
    stim_angle = np.array([0,45,90,135,180,225,270,315]) * np.pi/180
    mean_select=[]

    for n_it in range(n_iterations):
        all_sample1 = []
        
        np.random.seed(it_seed[n_it]) 
        for i_num,i_stim in enumerate(stim_num):
            all_sample1.append(np.random.choice(trials_fr[i_num], size=min_n_trials[i_num], replace=False, p=None))
        all_sample1 = np.concatenate(all_sample1)
       
        mean_resp = np.zeros((8,2))
        for i_num,i_stim in enumerate(stim_num):
            fr = np.random.choice(all_sample1, size=min_n_trials[i_num], replace=True, p=None)

            mean_resp[i_num] = circular_stats.mean_vector(fr, [stim_angle[i_num]]*len(fr))
        mean_select.append(circular_stats.mean_vector(mean_resp[:,0], mean_resp[:,1]) )
    mean_select=np.array(mean_select)
    return mean_select

In [8]:
def select_rand_trials_from_samples(trials_fr,min_n_trials,seed:int=1,n_iterations:int=1000):
    rng = np.random.default_rng(seed=seed)
    it_seed = rng.integers(low=1, high=10000, size=n_iterations, dtype=int)
    stim_num=['1','2','3','4','5','6','7','8']
    stim_angle = np.array([0,45,90,135,180,225,270,315]) * np.pi/180
    mean_select=[]
    for n_it in range(n_iterations):
        np.random.seed(it_seed[n_it]) 
        mean_resp = np.zeros((8,2))
        for i_num,i_stim in enumerate(stim_num):
            fr = np.random.choice(trials_fr[i_num], size=min_n_trials[i_num], replace=True, p=None)
            mean_resp[i_num] = circular_stats.mean_vector(fr, [stim_angle[i_num]]*len(fr))
        mean_select.append(circular_stats.mean_vector(mean_resp[:,0], mean_resp[:,1]) )
    mean_select=np.array(mean_select)
    return mean_select

### Read files

In [9]:
file1 = open("/envau/work/invibe/USERS/IBOS/code/flow/paths_bhv_lip.txt", "r")
lines_bhv = file1.readlines()
file1 = open("/envau/work/invibe/USERS/IBOS/code/flow/paths_sp_lip.txt", "r")
lines_sp = file1.readlines()
# load all  files
paths_bhv,paths_sp=[],[]
for line in lines_bhv:
    paths_bhv.append(line.strip())
for line in lines_sp:
    paths_sp.append(line.strip())

In [10]:
orient = 0 
color = 1
stim_num=['1','2','3','4','5','6','7','8']
sample=["o1_c1","o5_c1","o1_c5","o5_c5"]
palette = plt.get_cmap('hsv',64)
len_t = 450
n_iterations = 1000
win=100
step=1

In [15]:
selectivity_info: Dict[str, list] = defaultdict(list)
for n_bhv,n_sp in zip(paths_bhv[17:],paths_sp[17:]):
    s_path = os.path.normpath(n_sp).split(os.sep)
    date = s_path[-1][:19]
    data = SpikeData.from_python_hdf5(n_sp)
    bhv = BhvData.from_python_hdf5(n_bhv)
    task_corr_in, idx_tr_corr_in = get_task_variables(data,bhv,in_out='in')
    # Select data for the relevant trials
    test_stimuli = bhv.test_stimuli[idx_tr_corr_in]
    code_numbers = data.code_numbers[idx_tr_corr_in]
    code_samples = data.code_samples[idx_tr_corr_in]
    print(date)
    for i_neuron in range(len(data.clustersgroup)):
        sp_samples = data.sp_samples[idx_tr_corr_in,i_neuron]
        task_one_neuron=task_corr_in[task_corr_in['i_neuron']==i_neuron]
        task_trials = task_one_neuron['trial_idx'].values
        # select trials with at least one spike
        trial_idx = task_trials[np.nansum(sp_samples[task_trials],axis=1)>0]
        task_fr = task_one_neuron[np.in1d(task_one_neuron['trial_idx'] , trial_idx)]

        all_sample_feature = {"o1_c1":{"color":[],"orientation":[]},"o1_c5":{"color":[],"orientation":[]},"o5_c1":{"color":[],"orientation":[]},"o5_c5":{"color":[],"orientation":[]}}
        for i,sample in enumerate(["o1_c1","o1_c5","o5_c1","o5_c5"]):

            task_sample = task_fr[task_fr['sample']==sample]
            trials = task_sample['trial_idx'].values
            test_stimuli_samp = test_stimuli[trials]
            code_numbers_samp = code_numbers[trials]
            code_samples_samp = code_samples[trials]
            sp_samples_samp = sp_samples[trials]
            
            code = task_sample['code'].values
            test_stimuli_samp = delete_match(test_stimuli_samp,code)
            # use only the first n test stimuli presentations
            n_test = np.full(test_stimuli_samp.shape,np.nan)
            #n_test[:,:6] = test_stimuli_samp[:,:6]
            n_test = test_stimuli_samp
            n_test=n_test.astype(str)
            
            color_tests = get_sp_feature(sp_samples_samp,n_test,code_samples_samp,color_orient=color,stim_num=stim_num)
            orient_tests = get_sp_feature(sp_samples_samp,n_test,code_samples_samp,color_orient=orient,stim_num=stim_num)

            all_sample_feature[sample]["color"]= color_tests
            all_sample_feature[sample]["orientation"]= orient_tests

        o1_all = []
        o5_all = []
        c1_all = []
        c5_all = []
        for i in range(8):
            o1_all.append(np.concatenate([all_sample_feature["o1_c1"]['orientation'][i],all_sample_feature["o1_c5"]['orientation'][i]],axis=0))
            o5_all.append(np.concatenate([all_sample_feature["o5_c1"]['orientation'][i],all_sample_feature["o5_c5"]['orientation'][i]],axis=0))
            c1_all.append(np.concatenate([all_sample_feature["o1_c1"]['color'][i],all_sample_feature["o5_c1"]['color'][i]],axis=0))
            c5_all.append(np.concatenate([all_sample_feature["o1_c5"]['color'][i],all_sample_feature["o5_c5"]['color'][i]],axis=0))

        o1_trial_avg,o5_trial_avg,c1_trial_avg,c5_trial_avg=[],[],[],[]
        n_trials=[]
        for i in range(8):
            n_tr=[]
            avg = o1_all[i][:,100:350].mean(axis=1)*1000
            o1_trial_avg.append(avg)
            n_tr.append(len(avg))
            avg = o5_all[i][:,100:350].mean(axis=1)*1000
            o5_trial_avg.append(avg)
            n_tr.append(len(avg))
            avg = c1_all[i][:,100:350].mean(axis=1)*1000
            c1_trial_avg.append(avg)
            n_tr.append(len(avg))
            avg = c5_all[i][:,100:350].mean(axis=1)*1000
            c5_trial_avg.append(avg)
            n_tr.append(len(avg))
            n_trials.append(n_tr)

        min_n_trials = np.min(n_trials,axis=1).astype(int)
        all_select = []
        all_null=[]
        all_p_value=[]
        for trial_avg,feature in zip([o1_trial_avg,o5_trial_avg,c1_trial_avg,c5_trial_avg],['o1','o5','c1','c5']):

            if np.sum(min_n_trials<3)>0 or np.concatenate(trial_avg).sum()==0:
                p_value = np.nan
                mean_null = np.zeros((2,2))
                mean_select = np.zeros((2,2))
            else:
                mean_select = select_rand_trials_from_samples(trial_avg,min_n_trials,seed,n_iterations=1000)
                mean_null = get_null_vector_from_samples(trial_avg,min_n_trials,seed,n_iterations=1000)
                p_value = permutation_test(mean_select,mean_null)

            all_select.append(mean_select)
            all_null.append(mean_null)
            all_p_value.append(p_value)
            selectivity_info['date'] += [date]
            selectivity_info['i_neuron'] += [i_neuron]
            selectivity_info['neuron_type'] += [data.clustersgroup[i_neuron]]
            selectivity_info['p_value'] += [p_value]
            selectivity_info['sample1'] += [feature]

        if np.sum(np.array(all_p_value)<0.05) >0: 
        # fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(8,8),sharex=True,sharey=True,subplot_kw={'projection': 'polar'})
        # for i,(ax, feature)in enumerate(zip([ax1,ax2,ax3,ax4],['o1','o5','c1','c5'])):
        #     ax.set_rlabel_position(90)
        #     ax.scatter(all_null[i][:,1],all_null[i][:,0],label='null')
        #     ax.scatter(all_select[i][:,1],all_select[i][:,0],label='sample')
        #     ax.set_title('%s    p_value: %f'%(feature,all_p_value[i]) )
        # fig.suptitle('neuron: %d  date: %s'%(i_neuron,date),fontsize=7)
        # plt.legend(loc='upper right', bbox_to_anchor=(0, 0), prop={'size': 8})

            pref_dir_o1_0 = np.abs(0-all_select[0][:,1]*180/np.pi) 
            pref_dir_o5_0 = np.abs(0-all_select[1][:,1]*180/np.pi)
            diff=[]
            for i in range(len(pref_dir_o1_0)):
                shift = np.concatenate([pref_dir_o1_0[i:],pref_dir_o1_0[:i]])
                diff.append(shift-pref_dir_o5_0)
            diff=np.concatenate(diff)
            p_value_away_o = 1-np.sum(diff>0)/len(diff)
            p_value_toward_o = 1-np.sum(diff<0)/len(diff)
            pref_dir_c1_0 = np.abs(all_select[2][:,1]*180/np.pi - 0) 
            pref_dir_c5_0 = np.abs(all_select[3][:,1]*180/np.pi - 0)
            diff=[]
            for i in range(len(pref_dir_o1_0)):
                shift = np.concatenate([pref_dir_c1_0[i:],pref_dir_c1_0[:i]])
                diff.append(shift-pref_dir_c5_0)
            diff=np.concatenate(diff)
            p_value_away_c = 1-np.sum(diff>0)/len(diff)
            p_value_toward_c = 1-np.sum(diff<0)/len(diff)
            
            print(i_neuron)
            print(p_value_away_o)
            print(p_value_away_c)
    selectivity_info = pd.DataFrame(selectivity_info) 

2023-02-24_10-43-44


/tmp/ipykernel_43559/3123404539.py:7: RuntimeWarning: invalid value encountered in cast
  return test_stimuli.astype(str)


2023-02-28_10-15-02
5
0.806742
0.61924
2023-03-01_10-18-38
2023-03-03_10-59-32
2023-03-06_10-32-51
0
0.245016
0.072801
3
0.765489
0.45627700000000004
5
0.30971800000000005
0.73105
2023-03-07_10-14-11
0
0.535926
0.10825200000000001
1
0.26837
0.438875
2
0.632952
0.35619999999999996
2023-03-09_10-35-09
2023-03-10_10-30-26
3
0.411052
0.808952
2023-03-14_10-33-51
2023-03-16_10-20-01
2023-03-17_10-11-51
2
0.797627
0.15424800000000005
9
0.028000000000000025
0.797257
2023-03-20_10-39-08
2
0.844504
0.948214
3
0.28404300000000005
0.326384
5
0.82005
0.6583220000000001
10
0.875184
0.311072
2023-03-21_10-40-02
1
0.924053
0.453029
2023-03-22_10-34-47
2023-03-30_10-36-53
3
0.03600000000000003
0.761394
2023-10-06_10-38-57


IndexError: index 6 is out of bounds for axis 1 with size 5

In [14]:
selectivity_info1

,date,i_neuron,neuron_type,p_value,sample1
0,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3960,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,o1o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
1,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3960,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,o5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
2,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3960,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,c1o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
3,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3960,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
4,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3961,goodgoodgoodgoodgoodmuamuamuamuamuamuamuamuamu...,NaN,o1o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
5,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3961,goodgoodgoodgoodgoodmuamuamuamuamuamuamuamuamu...,NaN,o5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
6,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3961,goodgoodgoodgoodgoodmuamuamuamuamuamuamuamuamu...,NaN,c1o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
7,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3961,goodgoodgoodgoodgoodmuamuamuamuamuamuamuamuamu...,NaN,c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
8,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3962,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,o1o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
9,2022-11-22_10-59-032022-11-28_10-23-272022-11-...,3962,muagoodgoodgoodgoodmuamuamuamuamuamuamuamuamua...,NaN,o5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5c1c5o1o5...
